# Extract facts or cubes from XBRL reports
  1. install the open source Arelle processor and XULE plugin
  2. save output variables to a XULE expression and compile it as a .zip
  3. run the .zip with Arelle to extract numeric facts from an XBRL report to a file

## 1. Run the cell to install the Arelle processor and XULE plugin to extract data.

In [ ]:
# @title
import os, shutil, sys, site, platform
print('Please wait while Arelle, XULE and some helper packages are installed. \nA XULE version message appears below when the environment is ready.')

# In this example, Arelle and aniso are required to use XULE - get Arelle release details from GitHub (https://github.com/Arelle/arelle/releases).
# Use %pip -q install git+https://git@github.com/Arelle/arelle.git@master to use Arelle's development release
%pip -q install Arelle-release==2.37.6
%pip -q install aniso8601==9.0.1

# 1) locate Arelle's plugin directory (do not modify this location); remove temp and xuledir if they exist
plugindir = site.getsitepackages()[0] + '/arelle/plugin/'
edgardir = plugindir + 'EDGAR/'
xuledir = plugindir + 'xule/'
xodeldir = plugindir + 'xodel/'
serializerdir = plugindir + 'serializer/'
SimpleXBRLModeldir = plugindir + 'SimpleXBRLModel/'
edgartemp = plugindir + 'edgartemp/'
xtemp = plugindir + 'xtemp/'
if os.path.exists(xtemp):
  shutil.rmtree(edgardir)
  os.remove(plugindir + 'semanticHash.py')
  shutil.rmtree(xuledir)
  shutil.rmtree(xodeldir)
  shutil.rmtree(serializerdir)
  shutil.rmtree(SimpleXBRLModeldir)
  shutil.rmtree(xtemp)
else: ''
os.chdir(plugindir)

# 2) copy XULE and EDGAR plugins from GitHub to Arelle plugin directories
!git clone --quiet --depth=1 --branch 30050 --single-branch https://github.com/xbrlus/xule.git xtemp &> /dev/null
!git clone --quiet --depth=1 --branch 25.0.1 --single-branch https://github.com/Arelle/EDGAR.git edgartemp &> /dev/null
shutil.move(edgartemp, edgardir)
shutil.move(xtemp + 'plugin/semanticHash.py', plugindir)
shutil.move(xtemp + 'plugin/xule', xuledir)
shutil.move(xtemp + 'plugin/xodel', xodeldir)
shutil.move(xtemp + 'plugin/serializer', serializerdir)
shutil.move(xtemp + 'plugin/SimpleXBRLModel', SimpleXBRLModeldir)

# 3) confirm XULE (change -v to -h and re-run to see help contents for Arelle and XULE)
!arelleCmdLine --version
!arelleCmdLine --plugins 'EDGAR/transform' -v
!arelleCmdLine --plugins 'EDGAR/validate' -v
!arelleCmdLine --plugins 'xule' -v
print('\nArelle is installed and XULE is ready in ' + os.getcwd())

## 2. Run the cell to select a XULE expression that extracts facts or cubes and defines output variables. 
The XULE expression is saved to a file called 'extract-data.xule' and the compiled .zip is used by the Arelle processor in the next step.  

**Use 'Show code' in Colab to inspect XULE expression details and get details for extracting all facts (numeric and text)**

\* /content/ is Colab default

In [ ]:
# @title
# for mybinder.org, use /home/jovyan/ as location variable
output = 'fact' # @param ['fact', 'cube']
location = '/Users/stella/stella/Documents/PSU/Steve/BDC/Loop List/' # @param {type:"string"}
name = 'Extracted BDC-Ares-2024' # @param {type:"string"}
format = 'xlsx' # @param ['xlsx', 'json']
if format == 'xlsx':
  type = 'spreadsheet'
elif format == 'json':
  type = 'json'

# delete the prior version of XULE file if the name matches what was provided in 'with open' code below
curdir = os.getcwd()
if os.path.isfile(curdir + '/extract-data.xule'):
    os.remove(curdir + '/extract-data.xule')
    print('previous extract-data.xule deleted')
else: ''

# XULE facts expression starts after the ''' on the next line and continues to the second '''
xule_fact = '''
constant $filedir = '%s'
constant $filename = '%s'
constant $fileformat = '%s'
constant $filetype = '%s'
constant $extracted = $filedir + $filename + '.' + $fileformat

output-attribute file-location
output-attribute file-content
output-attribute file-append

output report-data

$report-facts = list({covered @})

$dict = dict(
  list('%s data',

    list(for $fact in filter $report-facts where $item.is-numeric
      list(
             $fact.concept.local-name,
             $fact.concept.name.namespace-uri,
             $fact.concept.label.text,
             $fact,
             $fact.concept.balance,
             $fact.concept.data-type.name,
             $fact.unit.string,
             $fact.concept.period-type,
             $fact.period.start,
             $fact.period.end,
             $fact.dimensions.join(', ','='),
             $fact.entity.id
      )
    ).sort('asc')
  )
)
$dataextract = if $filetype == 'spreadsheet'
    $dict.to-spreadsheet
    else $dict.to-json
$dataextract

file-content $rule-value
file-location $extracted
file-append true
'''

# XULE cubes expression starts after the ''' on the next line and continues to the second '''
xule_cube = '''
constant $filedir = '%s'
constant $filename = '%s'
constant $fileformat = '%s'
constant $filetype = '%s'
constant $extracted = $filedir + $filename + '.' + $fileformat

output-attribute file-location
output-attribute file-content
output-attribute file-append

output cube-data

$all-cubes = taxonomy().cubes

for $cube in $all-cubes
  $cube-facts = $cube.facts
  $dict = dict(
    list($cube.cube-concept.name.local-name,
      list(for $fact in $cube-facts
        list(
          $fact.concept.local-name,
          $fact.concept.name.namespace-uri,
          $fact.concept.label.text,
          $fact,
          $fact.concept.balance,
          $fact.concept.data-type.name,
          $fact.unit.string,
          $fact.concept.period-type,
          $fact.period.start,
          $fact.period.end,
          $fact.dimensions.join(', ','='),
          $fact.entity.id
        )
      )
    )
  )
$dataextract = if $filetype == 'spreadsheet'
    $dict.to-spreadsheet
    else $dict.to-json
$dataextract

file-content $rule-value
file-location $extracted
file-append true
'''

# save the selected XULE file and print a time stamp on the screen
!pip install tzlocal
with open('extract-data.xule', mode='w') as file:
    if output == 'fact':
        file.write(xule_fact % (location, name, format, type, name))
    elif output == 'cube':
        file.write(xule_cube % (location, name, format, type))

import datetime
from tzlocal import get_localzone
local_tz = get_localzone()
current_time = datetime.datetime.now(local_tz)
formatted_time = current_time.strftime('%H:%M:%S %Z on %b %d, %Y')
print('extract-data.xule updated ' + formatted_time + '\n' + output + ' data will be saved to ' + location + name + '.' + format + '\n\nXULE is compiling the file to .zip, so it can be executed in the next step.\n')

# variables for Arelle processing
FILE_NAME = 'extract-data.xule'
ZIP_NAME = 'extract-data.zip'
LOG_LOCATION = location + name + '-log.xml'

# compile XULE into .zip
!arelleCmdLine --plugins "xule" --xule-compile $FILE_NAME \
--xule-rule-set $ZIP_NAME --logFormat="[%(messageCode)s] %(message)s"

# copy XULE and .zip to location specified
shutil.copy(os.getcwd() + '/' + FILE_NAME, location + FILE_NAME)
shutil.copy(os.getcwd() + '/' + ZIP_NAME, location + ZIP_NAME)
print('\nThe XULE expression is compiled and ready.')


## 3. Loop over BDC CIKs and Filings to Execute XULE Extraction

This script extends the XBRL US sample workflow by replacing the single sample filing URL with a loop over BDC CIKs and their filings within the specified "DATE_RANGE".
 Modified from the XBRL US sample code:
 - Replaced single sample URL with a loop over BDC CIKs and filings.
 - The final Arelle execution step remains the same with the original XBRL US example.


In [ ]:
from edgar import Company, set_identity
import os
import time
import re

# the output file path
BASE_PATH = "/Users/stella/stella/Documents/PSU/Steve/BDC/Loop List/"

# request header for SEC API
set_identity("Stella Xie zfx5135@psu.edu") 

# CIK from SEC BDC file
raw_ciks = [
17313,2008748,1786108,1675033,1513363,1794776,
1508655,1655050,81955,1370755,1932591,1832148,
1743415,1418076,1925309,1834543,1902649,1726548,
1278752,1916099,1998387,1372807,1571329,1572694,
1860424,2011498,1837532,1849894,1633336,1646614,
1715933,1825265,1916608,1784700,1653384,1772704,
1899996,1674760,2020354,1865174,1948368,1920453,
1841514,1578348,1603480,1911321,2006758,1920145,
1954360,1959604,1959568,1890107,1993402,1987221,
1634452,1982701,1899017,1965934,1504619,1383414,
1843162,1786835,1143513,1950976,1988280,1905824,
1950803,1588272,1377936,1259429,1817825,1985375,
1779523,845385,1747172,1850787,1922947,1490349,
1280784,1287032,1287750,1321741,1999538,1379785,
1396440,1618697,1618694,1618696,1895316,1979306,
1923622,1825384,1633858,1414932,1487428,1495584,
1476765,1487918,1509470,1496099,1512931,1521945,
1534254,1535778,1544206,1552198,1551901,1901037,
1870267,1580345,1950572,1422183,1737924,1911066,
1930679,1975736,1501729,1655887,1655888,1661306,
1702510,1717310,1736035,1742313,1747777,1766037,
1781870,1782524,1792509,1803498,1807427,1811972,
1812554,1825248,1838126,1849089,1850938,1851277,
1851322,1859919,1868878,1869453,1872371,1885968,
1889668,1901164,1901606,1901612,1913724,1918712,
1919369,1925531,1930087,1944831,1955010,1973476,
1974793,1976719,1983514,1987731,1989817,2012839,
2018545,1976336,2031283,2031750,2028686,2037804
]

bdc_ciks = [str(cik).zfill(10) for cik in raw_ciks]
   
#filing type
FORMS = ["10-K", "10-Q", "10-K/A", "10-Q/A"] 
DATE_RANGE = "2024-01-01:2024-12-31"

#the first loop for BDCs cik 
for cik10 in bdc_ciks:

    company = Company(cik10)
    filings = company.get_filings(form=FORMS).filter(date=DATE_RANGE)
    
    #the second loop for each BDC filings
    for filing in filings:

        accession = filing.accession_number.replace("-", "")

        # getting URL from SEC filing
        accession_path = filing.accession_number.replace("-", "")
        reportfile = f"https://www.sec.gov/Archives/edgar/data/{int(cik10)}/{accession_path}/{filing.primary_document}"

        print("Running:", accession)
        print("URL:", reportfile)

        # reading Step2 xule
        with open(BASE_PATH + "extract-data.xule", "r") as f:
            xule_text = f.read()

        # creating filename
        new_filename = f"CIK{cik10}_{accession}"
        xule_text = re.sub(
            r"constant \$filename = '.*?'",
            f"constant $filename = '{new_filename}'",
            xule_text
        )

        # temporary XULE (XULE rule file from step 2)
        temp_xule = BASE_PATH + "temp-extract.xule"
        temp_zip  = BASE_PATH + "temp-extract.zip"

        with open(temp_xule, "w") as f:
            f.write(xule_text)

        # recompile XULE rule for Arelle 
        os.system(
            f'arelleCmdLine --plugins "xule" '
            f'--xule-compile "{temp_xule}" '
            f'--xule-rule-set "{temp_zip}"'
        )

        # execute (same as sample code)
        os.system(
            f'arelleCmdLine --plugins "xule|EDGAR/transform|validate/EFM" '
            f'--xule-rule-set "{temp_zip}" '
            f'-v -f "{reportfile}" '
            f'--noCertificateCheck'
        )

        time.sleep(1)
